In [1]:
from data_manager import DataManager

#ds_path = "/home/gwydion/SSC/cebra/mobie-data-testing/data/cebra_em_example/seg_er_5nm_mito_10nm/CebraEM/images/bdv-n5"
ds_path = "/home/gwydion/SSC/cebra/mobie-data-testing/data/cebra_em_example/seg_er_5nm_mito_10nm/CebraEM"

mito_filename = "mito-it00_b0_6_stitched.n5"
er_filename = "er-it00_b0_5_stitched"
mem_filename = "em-membrane_prediction"
test = "test"

scaling_factor = 3
original_resolution_er=(5,5,5)
original_resolution_mito=(10,10,10)
data_key = f"setup0/timepoint0/s{scaling_factor}"


dm3 =  DataManager(ds_path)
dm3.load_organelle_data(mito_filename, "mito", data_key=data_key, original_resolution=original_resolution_mito, scaling_factor=scaling_factor)
dm3.load_organelle_data(er_filename, "er", data_key=data_key, original_resolution=original_resolution_er, scaling_factor=scaling_factor)
#dm3.load_organelle_data(mem_filename, "mem", data_key=data_key, original_resolution=original_resolution, scaling_factor=scaling_factor)

INFO:root:Loading dataset from: /home/gwydion/SSC/cebra/mobie-data-testing/data/cebra_em_example/seg_er_5nm_mito_10nm/CebraEM
INFO:root:Available segmentations are: ['em-membrane_prediction', 'em-raw', 'em-supervoxels', 'er-it00_b0_5_stitched', 'mito-it00_b0_6_stitched']
INFO:root:Successfully read mito-it00_b0_6_stitched.
INFO:root:Successfully read er-it00_b0_5_stitched.


The distance matrix gives the closest distance between each object and any other object.
Since this is mesh based it takes a long time on higher resolutions.
This matrix is later used to filter close neighbors

In [2]:
dist_matrix = dm3.calculate_distance_matrix()
# this function can throw quite a lot of warnings, but works fine

/home/gwydion/mambaforge/envs/mobie-testing/lib/python3.9/site-packages/skimage/measure/_regionprops.py:433: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1891856420  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 13  Error-roundoff 1.7e-14  _one-merge 1.2e-13  _near-inside 6e-13
  Visible-distance 3.4e-14  U-max-coplanar 3.4e-14  Width-outside 6.8e-14
  _wide-facet 2e-13  _maxoutside 1.4e-13

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p9(v4):     3     3    13
- p1(v3):     1     1     0
- p10(v2):     4     4     7
- p0(v1):     0     0     1

The center point is coplanar with a facet, or a vertex is coplanar
with a neigh

KeyboardInterrupt: 

In [ ]:
dm3.distance_df.head()

Label,er_816,er_1978,er_2315,er_2473,er_2897,er_3018,er_3171,er_3228,er_3251,er_3264,...,er_7907,er_8270,er_8272,er_8273,er_8678,er_8687,er_8751,er_9311,er_9492,er_9564
Label,,,,,,,,,,,,,,,,,,,,,
er_816,0.000000,2320.000000,2581.162529,2496.717845,5888.565190,5297.546602,2818.226393,4076.861538,2938.298828,3215.960199,...,5702.490684,4443.782173,3807.361291,3847.492690,5675.491168,4832.556259,5069.753446,4423.211503,56.568542,1787.064632
er_1978,2320.000000,0.000000,4994.076491,4943.197346,6533.605437,6004.265151,1417.603612,2766.658635,1046.135746,1715.808847,...,6731.894236,4679.829057,3916.733333,4101.901998,5412.282328,3943.602414,4489.276111,4698.254995,1043.072385,2485.155931
er_2315,2581.162529,4994.076491,0.000000,924.337601,5789.922279,5242.289576,5143.073011,6114.638174,5362.984244,5636.452785,...,5077.952343,5014.538862,4787.316576,4637.930573,6796.705084,6739.970326,6741.869177,5377.285561,113.137085,3257.483691
er_2473,2496.717845,4943.197346,924.337601,0.000000,6251.783745,5571.355311,4926.986909,5811.437000,5108.737613,5290.897844,...,5435.880793,5792.132595,5340.861354,5324.058602,6284.457017,6272.989718,6136.839578,4787.316576,0.000000,3564.267106
er_2897,5888.565190,6533.605437,5789.922279,6251.783745,0.000000,430.813185,4773.258845,4763.192207,5310.216568,5682.253074,...,113.137085,2701.110883,2053.679624,3019.933774,4987.023160,5230.678732,5447.641692,4793.328697,80.000000,3775.288068


In [ ]:
import plotly.express as px
fig = px.imshow(dist_matrix, labels={"color": "distance"})
fig.show()

here i calculated the volume each organell is taking from the entire cell block

In [ ]:
# find percentage of volume taken by each organelle
import numpy as np
print("total volume:", dm3.total_volume)
# note that the mito volume is not the same as the  er volume
# it seems the mito has half as many data points, although they should have the same scaling factor.
# so these results are completly useless
df = dm3.analysis_results
for org_type in np.unique(df["org_type"]):
    percentage = df.loc[df["org_type"] == org_type, "volume_voxels"].sum() / dm3.total_volume[org_type]
    print(org_type, f"{round(percentage*100, 2)}%")

total volume: {'er': 224755712000}
er 4.52%


In [ ]:
dm3.find_neighbours(50,0)

defaultdict(list,
            {'er_2473': ['er_9492'],
             'er_3228': ['er_9492'],
             'er_5906': ['er_9492'],
             'er_7172': ['er_9492'],
             'er_7460': ['er_9492'],
             'er_8270': ['er_9492'],
             'er_8272': ['er_9492'],
             'er_9311': ['er_9492'],
             'er_9492': ['er_2473',
              'er_3228',
              'er_5906',
              'er_7460',
              'er_8270',
              'er_8272',
              'er_9311',
              'er_7172']})

# TODO 
extend mesh in all direction

In [ ]:
dm3.analysis_results.head()

,label,org_type,volume_voxels,bbox,bbox_dim_nm,bbox_vol_nm,centroid,solidity,all_coords,surf_area_mesh_nm,volume_mesh,water_tight,sphericity,flatness_ratio,num_neighbours_50-0
Label,,,,,,,,,,,,,,,
er_816,er_816,er,5120000,"[[69, 36, 42], [76, 38, 45]]","[35, 10, 15]",5250,"(5784.0, 2888.0, 3400.0)",0.588235,"[[69, 36, 42], [70, 36, 42], [71, 36, 42], [71...",1.563250e+05,5.474133e+07,False,4.459978,0.389708,NaN
er_1978,er_1978,er,5120000,"[[69, 8, 17], [75, 15, 23]]","[30, 35, 30]",31500,"(5648.0, 880.0, 1576.0)",0.357143,"[[69, 11, 20], [69, 12, 20], [69, 14, 22], [70...",1.551700e+05,2.261333e+06,True,0.536932,0.847134,NaN
er_2315,er_2315,er,6144000,"[[64, 66, 58], [73, 71, 63]]","[45, 25, 25]",28125,"(5486.666666666667, 5386.666666666667, 4740.0)",0.342857,"[[64, 66, 62], [64, 67, 62], [67, 67, 60], [68...",2.138926e+05,3.072000e+06,True,0.477789,0.754476,NaN
er_2473,er_2473,er,45568000,"[[59, 53, 70], [76, 65, 76]]","[85, 60, 30]",153000,"(5237.752808988764, 4421.573033707865, 5794.15...",0.211401,"[[59, 64, 75], [60, 64, 75], [61, 55, 73], [61...",1.015939e+06,4.949333e+06,False,0.138244,0.253425,1.0
er_2897,er_2897,er,5632000,"[[5, 65, 10], [9, 71, 16]]","[20, 30, 30]",18000,"(487.27272727272725, 5454.545454545455, 952.72...",0.234043,"[[5, 65, 12], [5, 66, 12], [5, 67, 12], [5, 68...",1.640372e+05,1.962667e+06,True,0.462139,0.909091,NaN


In [ ]:
#try this
# skimage.measure.intersection_coeff

a voxel based approach would most likely be much much quicker.


In [ ]:
ds_mito_15 = dm3._filter_ds("mito_15", "mito")
ds_er_7706 = dm3._filter_ds("er_7706", "er")
ds_er_7727 = dm3._filter_ds("er_7727", "er")

In [ ]:
coords_ds_mito_15 = dm3.analysis_results.loc["mito_15"]["all_coords"]
coords_ds_er_7706 = dm3.analysis_results.loc["er_7706"]["all_coords"]

this might actually work

In [ ]:
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.neighbors import BallTree

voxels1 = np.argwhere(coords_ds_mito_15)
voxels2 = np.argwhere(coords_ds_er_7706)

tree = BallTree(voxels1)
# coordinates are not yet scaled to nm.
radius = 2
indices = tree.query_radius(voxels2, radius)
# i think this is what we want, but i'm not sure how to interpret it yet.
indices

array([array([0, 1, 2, 3, 4, 6]), array([0, 1, 3, 4, 7]),
       array([0, 2, 3, 4, 5, 6, 8]), array([0, 1, 2, 3, 4, 5, 6, 7, 9]),
       array([ 1,  2,  3,  4,  6,  7, 10]),
       array([ 0,  2,  3,  5,  6,  7,  8,  9, 11]),
       array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 12]),
       array([ 1,  3,  4,  5,  6,  7,  9, 10, 13]),
       array([ 2,  5,  6,  8,  9, 10, 11, 12, 14]),
       array([ 3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15]),
       array([ 4,  6,  7,  8,  9, 10, 12, 13, 16]),
       array([ 5,  8,  9, 11, 12, 13, 14, 15, 17]),
       array([ 6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 18]),
       array([ 7,  9, 10, 11, 12, 13, 15, 16, 19]),
       array([ 8, 11, 12, 14, 15, 16, 17, 18, 20]),
       array([ 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21]),
       array([10, 12, 13, 14, 15, 16, 18, 19, 22]),
       array([11, 14, 15, 17, 18, 19, 20, 21, 23]),
       array([12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24]),
       array([13, 15, 16, 17, 18, 19, 21, 22, 25]),
       ar

In [ ]:

# Compute the distance between each pair of voxels
distances = cdist(voxels1, voxels2)

# Get the pairs of voxels that are under a certain distance from each other
threshold = 2  # Set your distance threshold here
close_pairs = np.where(distances < threshold)
close_pairs

(array([ 0,  0,  0,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
         3,  3,  3,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
         8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
        11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13,
        13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 18, 18, 18,
        18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20,
        20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 23,
        23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25,
        25, 25, 25, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 28, 28, 28, 28, 28, 28, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32,
        33, 